In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import seed
seed(1)
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import tensorflow
tensorflow.random.set_seed(1)
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import keras.backend as K

In [ ]:
df = pd.read_csv(r"C:\Users\Avenger\Desktop\ML Assignment 3\Mouse-annotation\labeled-data\0026\A20490554_CollectedData_annotation.csv")
df


# Data Cleaning

In [ ]:
df = df.drop(columns=['scorer', 'Unnamed: 1', 'Unnamed: 2'])
df.rename(columns={'annotation': 'mouse1 topleft x', 'annotation.1': 'mouse1 topleft y',
                   'annotation.2': 'mouse1 rightdown x', 'annotation.3': 'mouse1 rightdown y',
                   'annotation.4': 'mouse1 nose x', 'annotation.5': 'mouse1 nose y',
                   'annotation.6': 'mouse1 leftear x', 'annotation.7': 'mouse1 leftear y',
                   'annotation.8': 'mouse1 rightear x', 'annotation.9': 'mouse1 rightear y',
                   'annotation.10': 'mouse1 leftHip x', 'annotation.11': 'mouse1 leftHip y',
                   'annotation.12': 'mouse1 rightHip x', 'annotation.13': 'mouse1 rightHip y',
                   'annotation.14': 'mouse1 tailBase x', 'annotation.15': 'mouse1 tailBase y',
                   'annotation.16': 'mouse1 tailEnd x', 'annotation.17': 'mouse1 tailEnd y',
                   'annotation.18': 'mouse2 topleft x', 'annotation.19': 'mouse2 topleft y',
                   'annotation.20': 'mouse2 rightdown x', 'annotation.21': 'mouse2 rightdown y',
                   'annotation.22': 'mouse2 nose x', 'annotation.23': 'mouse2 nose y',
                   'annotation.24': 'mouse2 leftear x', 'annotation.25': 'mouse2 leftear y',
                   'annotation.26': 'mouse2 rightear x', 'annotation.27': 'mouse2 rightear y',
                   'annotation.28': 'mouse2 leftHip x', 'annotation.29': 'mouse2 leftHip y',
                   'annotation.30': 'mouse2 rightHip x', 'annotation.31': 'mouse2 rightHip y',
                   'annotation.32': 'mouse2 tailBase x', 'annotation.33': 'mouse2 tailBase y',
                   'annotation.34': 'mouse2 tailEnd x', 'annotation.35': 'mouse2 tailEnd y'}, inplace=True)

df

In [ ]:
df = df.drop(range(0,3))
df = df.reset_index(drop=True)
df

In [ ]:
df.info()

In [ ]:
# converting data types to float
df = df.apply(pd.to_numeric)
df.info()

In [ ]:
df

# Ground Truth

In [ ]:
df['mouse1 center x'] = (df['mouse1 topleft x'] + df['mouse1 rightdown x'])/2
df['mouse1 center y'] = (df['mouse1 topleft y'] + df['mouse1 rightdown y'])/2
df['mouse2 center x'] = (df['mouse2 topleft x'] + df['mouse2 rightdown x'])/2
df['mouse2 center y'] = (df['mouse2 topleft y'] + df['mouse2 rightdown y'])/2

In [ ]:
df1 = df
df3 = df

# MOUSE 1

# Handeling Null values 

# Filling null values with 0

In [ ]:
df=df.fillna(0)
df2 = df
df

In [ ]:
df


# Labeled data for training

In [ ]:
y1 = np.column_stack((df["mouse1 center x"], df["mouse1 center y"]))


In [ ]:
x1 = np.column_stack((df['mouse1 nose x'],df['mouse1 nose y'],
                      df['mouse1 leftear x'],df['mouse1 leftear y'],
                      df['mouse1 rightear x'],df['mouse1 rightear y'],
                      df['mouse1 leftHip x'],df['mouse1 leftHip y'],
                      df['mouse1 rightHip x'],df['mouse1 rightHip y'],
                      df['mouse1 tailBase x'],df['mouse1 tailBase y'],
                      df['mouse1 tailEnd x'],df['mouse1 tailEnd y']))

In [ ]:
x1 = sm.add_constant(x1, prepend=True)

# Splitting the data into training and testing set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x1, y1, test_size = 0.3)

In [ ]:
my_test = y_test

# Feature Scaling Function

In [ ]:
def featureScaling(X_train, X_test, y_train, y_test):
    y_train=np.reshape(y_train, (-1,1))
    y_test=np.reshape(y_test, (-1,1))
    scaler_x = MinMaxScaler()
    global scaler_y 
    scaler_y = MinMaxScaler()
    print(scaler_x.fit(X_train))
    xtrain_scale=scaler_x.transform(X_train)
    print(scaler_x.fit(X_test))
    xval_scale=scaler_x.transform(X_test)
    print(scaler_y.fit(y_train))
    ytrain_scale=scaler_y.transform(y_train)
    print(scaler_y.fit(y_test))
    yval_scale=scaler_y.transform(y_test)
    return (xtrain_scale, xval_scale, ytrain_scale, yval_scale) 

# LSTM Model

In [ ]:
def custom_loss(y_true, y_pred):
    return K.sum((y_true - y_pred)**2)
def lstm(xtrain_scale, ytrain_scale, xval_scale,input_layer,hidden_layer1,hidden_layer2,hidden_layer3,output_layer):
    model = Sequential()
    model.add(Dense(input_layer, input_dim=input_layer, kernel_initializer='normal', activation='relu'))
    model.add(Dense(hidden_layer1, activation='relu'))
    Dropout(0.2)
    model.add(Dense(hidden_layer2, activation='relu'))
    Dropout(0.2)
    model.add(Dense(hidden_layer3, activation='relu'))
    Dropout(0.2)
    model.add(Dense(output_layer, activation='linear'))
    print(model.summary())
    model.compile(loss=custom_loss, optimizer='adam', metrics=[custom_loss])
    history=model.fit(xtrain_scale, ytrain_scale, epochs=60, batch_size=150, verbose=1, validation_split=0.3)
    predictions = model.predict(xval_scale)
    return (history, predictions)

# Model Loss Plot function

In [ ]:
def plotting(history):
    print(history.history.keys())
    # "Loss"
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

# Mean Squared error and l2 loss function

In [ ]:
def loss(my_test, predictions):
    print("Mean Squared Error is: " + str(mean_squared_error(my_test, predictions)))
    print("L2 loss: "+ str(np.sum(np.power((my_test-predictions),2))))
    return (str(np.sum(np.power((my_test-predictions),2))))

# Feature Scaling

In [ ]:
xtrain_scale, xval_scale, ytrain_scale, yval_scale = featureScaling(X_train, X_test, y_train, y_test)

In [ ]:
history, predictions = lstm(xtrain_scale, ytrain_scale, xval_scale,15,1496, 500, 180,2)

# Model Loss plot

In [ ]:
plotting(history)

# Predictions

In [ ]:
predictions = scaler_y.inverse_transform(predictions)
predictions

# Mean Squared error and l2 loss

In [ ]:
loss_mouse_1_null_0 = loss(my_test, predictions)

# MOUSE 2

# Labeled data for training

In [ ]:
y2 = np.column_stack((df["mouse2 center x"], df["mouse2 center y"]))

In [ ]:
x2 = np.column_stack((df['mouse2 nose x'],df['mouse2 nose y'],
                      df['mouse2 leftear x'],df['mouse2 leftear y'],
                      df['mouse2 rightear x'],df['mouse2 rightear y'],
                      df['mouse2 leftHip x'],df['mouse2 leftHip y'],
                      df['mouse2 rightHip x'],df['mouse2 rightHip y'],
                      df['mouse2 tailBase x'],df['mouse2 tailBase y'],
                      df['mouse2 tailEnd x'],df['mouse2 tailEnd y']))

In [ ]:
x2 = sm.add_constant(x1, prepend=True)

# Splitting the data into training and testing set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x2, y2, test_size = 0.3)

In [ ]:
my_test = y_test

# Feature Scaling

In [ ]:
xtrain_scale, xval_scale, ytrain_scale, yval_scale = featureScaling(X_train, X_test, y_train, y_test)

In [ ]:
history, predictions = lstm(xtrain_scale, ytrain_scale, xval_scale,15,1496, 500, 180,2)

# Model Loss plot

In [ ]:
plotting(history)

# Predictions

In [ ]:
predictions = scaler_y.inverse_transform(predictions)
predictions

# Mean Squared error and l2 loss

In [ ]:
loss_mouse_2_null_0 = loss(my_test, predictions)

# Second Approach (Dropping the rows containing null values)

In [ ]:
df1=df1.dropna()
df1

# Labeled data for training

# Model for Mouse 1

# Labeled data for training

In [ ]:
y1 = np.column_stack((df1["mouse1 center x"], df1["mouse1 center y"]))

In [ ]:
x1 = np.column_stack((df1['mouse1 nose x'],df1['mouse1 nose y'],
                      df1['mouse1 leftear x'],df1['mouse1 leftear y'],
                      df1['mouse1 rightear x'],df1['mouse1 rightear y'],
                      df1['mouse1 leftHip x'],df1['mouse1 leftHip y'],
                      df1['mouse1 rightHip x'],df1['mouse1 rightHip y'],
                      df1['mouse1 tailBase x'],df1['mouse1 tailBase y'],
                      df1['mouse1 tailEnd x'],df1['mouse1 tailEnd y']))

In [ ]:
x1 = sm.add_constant(x1, prepend=True)

# Splitting the data into training and testing set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x1, y1, test_size = 0.3)

In [ ]:
my_test = y_test

# Feature Scaling

In [ ]:
xtrain_scale, xval_scale, ytrain_scale, yval_scale = featureScaling(X_train, X_test, y_train, y_test)

In [ ]:
history, predictions = lstm(xtrain_scale, ytrain_scale, xval_scale,15,1496, 500, 180,2)

# Model Loss plot

In [ ]:
plotting(history)

# Predictions

In [ ]:
predictions = scaler_y.inverse_transform(predictions)
predictions

# Mean Squared error and l2 loss

In [ ]:
loss_mouse_1_null_drop = loss(my_test, predictions)

# Model for Mouse 2

# Labeled data for training

In [ ]:
y2 = np.column_stack((df1["mouse2 center x"], df1["mouse2 center y"]))

In [ ]:
x2 = np.column_stack((df1['mouse2 nose x'],df1['mouse2 nose y'],
                      df1['mouse2 leftear x'],df1['mouse2 leftear y'],
                      df1['mouse2 rightear x'],df1['mouse2 rightear y'],
                      df1['mouse2 leftHip x'],df1['mouse2 leftHip y'],
                      df1['mouse2 rightHip x'],df1['mouse2 rightHip y'],
                      df1['mouse2 tailBase x'],df1['mouse2 tailBase y'],
                      df1['mouse2 tailEnd x'],df1['mouse2 tailEnd y']))

In [ ]:
x2 = sm.add_constant(x2, prepend=True)

# Splitting the data into training and testing set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x2, y2, test_size = 0.3)

In [ ]:
my_test = y_test

# Feature Scaling

In [ ]:
xtrain_scale, xval_scale, ytrain_scale, yval_scale = featureScaling(X_train, X_test, y_train, y_test)

In [ ]:
history, predictions = lstm(xtrain_scale, ytrain_scale, xval_scale,15,1496, 500, 180,2)

# Model Loss plot

In [ ]:
plotting(history)

# Predictions

In [ ]:
predictions = scaler_y.inverse_transform(predictions)
predictions

# Mean Squared error and l2 loss

In [ ]:
loss_mouse_2_null_drop = loss(my_test, predictions)

# Combined model for mouse 1 and mouse 2

# Using dataframe with null values as 0

In [ ]:
df2

In [ ]:
y12 = np.column_stack((df2["mouse1 center x"], df2["mouse1 center y"], df2["mouse2 center x"], df2["mouse2 center y"]))

In [ ]:
x12 = np.column_stack((df2['mouse1 nose x'],df2['mouse1 nose y'],
                      df2['mouse1 leftear x'],df2['mouse1 leftear y'],
                      df2['mouse1 rightear x'],df2['mouse1 rightear y'],
                      df2['mouse1 leftHip x'],df2['mouse1 leftHip y'],
                      df2['mouse1 rightHip x'],df2['mouse1 rightHip y'],
                      df2['mouse1 tailBase x'],df2['mouse1 tailBase y'],
                      df2['mouse1 tailEnd x'],df2['mouse1 tailEnd y'],
                      df2['mouse2 nose x'],df2['mouse2 nose y'],
                      df2['mouse2 leftear x'],df2['mouse2 leftear y'],
                      df2['mouse2 rightear x'],df2['mouse2 rightear y'],
                      df2['mouse2 leftHip x'],df2['mouse2 leftHip y'],
                      df2['mouse2 rightHip x'],df2['mouse2 rightHip y'],
                      df2['mouse2 tailBase x'],df2['mouse2 tailBase y'],
                      df2['mouse2 tailEnd x'],df2['mouse2 tailEnd y']))

In [ ]:
x12 = sm.add_constant(x12, prepend=True)

# Splitting the data into training and testing set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x12, y12, test_size = 0.3,random_state=0)

In [ ]:
my_test = y_test

# Feature Scaling

In [ ]:
xtrain_scale, xval_scale, ytrain_scale, yval_scale = featureScaling(X_train, X_test, y_train, y_test)

In [ ]:
history, predictions = lstm(xtrain_scale, ytrain_scale, xval_scale,29,2992, 5000, 1000,4)

# Model Loss plot

In [ ]:
plotting(history)

# Predictions

In [ ]:
predictions = scaler_y.inverse_transform(predictions)
predictions

# Mean Squared error and l2 loss

In [ ]:
loss_mouse_1_mouse_2_null_0 = loss(my_test, predictions)

# Combined Model for Mouse 1 and Mouse 2 

# Fill null values with column means

In [ ]:
df3

In [ ]:
colHeader = list(df3.columns.values)

In [ ]:
# getting all the columns 
colHeader

# Storing column means

In [ ]:
def getMean(col):
    mean_value=df3[col].mean()
    df3[col].fillna(value=mean_value, inplace=True)
for i in range (0,len(colHeader)):
    getMean(colHeader[i])

In [ ]:
df3

In [ ]:
y12 = np.column_stack((df3["mouse1 center x"], df3["mouse1 center y"], df3["mouse2 center x"], df3["mouse2 center y"]))

In [ ]:
x12 = np.column_stack((df3['mouse1 nose x'],df3['mouse1 nose y'],
                      df3['mouse1 leftear x'],df3['mouse1 leftear y'],
                      df3['mouse1 rightear x'],df3['mouse1 rightear y'],
                      df3['mouse1 leftHip x'],df3['mouse1 leftHip y'],
                      df3['mouse1 rightHip x'],df3['mouse1 rightHip y'],
                      df3['mouse1 tailBase x'],df3['mouse1 tailBase y'],
                      df3['mouse1 tailEnd x'],df3['mouse1 tailEnd y'],
                      df3['mouse2 nose x'],df3['mouse2 nose y'],
                      df3['mouse2 leftear x'],df3['mouse2 leftear y'],
                      df3['mouse2 rightear x'],df3['mouse2 rightear y'],
                      df3['mouse2 leftHip x'],df3['mouse2 leftHip y'],
                      df3['mouse2 rightHip x'],df3['mouse2 rightHip y'],
                      df3['mouse2 tailBase x'],df3['mouse2 tailBase y'],
                      df3['mouse2 tailEnd x'],df3['mouse2 tailEnd y']))

In [ ]:
x12 = sm.add_constant(x12, prepend=True)

# Splitting the data into training and testing dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x12, y12, test_size = 0.3,random_state=0)

In [ ]:
my_test = y_test

# Feature Scaling

In [ ]:
xtrain_scale, xval_scale, ytrain_scale, yval_scale = featureScaling(X_train, X_test, y_train, y_test)

In [ ]:
history, predictions = lstm(xtrain_scale, ytrain_scale, xval_scale,29,2992, 5000, 1000,4)

# Model Loss plot

In [ ]:
plotting(history)

# Predictions

In [ ]:
predictions = scaler_y.inverse_transform(predictions)
predictions

# Mean Squared error and l2 loss

In [ ]:
loss_mouse_1_mouse_2_null_column_mean = loss(my_test, predictions)

# Final L2 losses for 6 experiments performed

In [ ]:
print("L2 Loss of mouse 1 with null values as 0: " + loss_mouse_1_null_0)
print("L2 Loss of mouse 2 with null values as 0: " + loss_mouse_2_null_0)
print("L2 Loss of mouse 1 after dropping null rows: " + loss_mouse_1_null_drop)
print("L2 Loss of mouse 2 after dropping null rows: " + loss_mouse_2_null_drop)
print("L2 Loss of mouse 1 and mouse 2 with null values as 0: " + loss_mouse_1_mouse_2_null_0)
print("L2 Loss of mouse 1 and mouse 2 after filling the null values with column mean: " + loss_mouse_1_mouse_2_null_column_mean)